In [141]:
from dash import Dash, html, dcc, Output, Input
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
from pathlib import Path 
from datetime import datetime
import plotly.figure_factory as ff
from plotly import graph_objects as go
import numpy as np


In [142]:
df_defender = pd.read_csv('../../data/Stats_csv/Processed_Defender_data.csv')
df_forward = pd.read_csv('../../data/Stats_csv/Processed_Forward_data.csv')
df_goalkeeper = pd.read_csv('../../data/Stats_csv/Processed_Goalkeeper_data.csv')
df_midfielder = pd.read_csv('../../data/Stats_csv/Processed_Midfielder_data.csv')

Finding Distribution and the Average of Age of Players

In [143]:
today = datetime.now() 

df_defender['Date of Birth'] = pd.to_datetime(df_defender['Date of Birth'], dayfirst=True)
df_defender['Age'] = today.year - df_defender['Date of Birth'].dt.year

df_forward['Date of Birth'] = pd.to_datetime(df_forward['Date of Birth'], dayfirst=True)
df_forward['Age'] = today.year - df_forward['Date of Birth'].dt.year

df_goalkeeper['Date of Birth'] = pd.to_datetime(df_goalkeeper['Date of Birth'], dayfirst=True)
df_goalkeeper['Age'] = today.year - df_goalkeeper['Date of Birth'].dt.year

df_midfielder['Date of Birth'] = pd.to_datetime(df_midfielder['Date of Birth'], dayfirst=True)
df_midfielder['Age'] = today.year - df_midfielder['Date of Birth'].dt.year

hist_data = (
    df_defender['Age'].tolist() + 
    df_forward['Age'].tolist() + 
    df_goalkeeper['Age'].tolist() + 
    df_midfielder['Age'].tolist()
)

group = ["All Players' Age"]
fig_age = ff.create_distplot([hist_data], group,bin_size=1, show_rug=False)
fig_age.update_layout(bargap=0.01, title="Age Distribution of All Players",xaxis_title = 'Age', yaxis_title = 'Percentage',showlegend = False)
fig_age.update_yaxes(
    tickformat=".1%",  # Show percentage format
    title_text="Percentage"
)
fig_age.show() 

fig_age2 = px.pie()

In [144]:
df = pd.concat([df_defender,df_forward,df_goalkeeper,df_midfielder],ignore_index=True)
age_avg=df['Age'].mean()
fig = px.violin(df, y="Age", x="Position", box=True, title='Players Age distribution by position',template='plotly')
fig.add_shape(type="line", line_color="blue", line_width=3, opacity=1, line_dash="dot",x0=0, x1=1, xref="paper", y0=age_avg, y1=age_avg, yref="y")
fig.show()

Chỉ số của từng cầu thủ.

In [145]:
dselected_columns = ['Passes per match','Recoveries','Duels won','Tackles','Interceptions','Clearances']
fselected_columns = ['Goals','Assists','Big Chances Created','Shots on target','Interceptions','Tackles']
gselected_columns = ['Saves','Clean cheets','Goals conceded','Accurate long balls','Penalties saved','Errors leading to goal']
mselected_columns = ['Goals','Assists','Duels won','Tackles','Big Chances Created','Passes per match']

def radar(s:str):
    df_all = pd.concat([df_defender,df_forward,df_goalkeeper,df_midfielder],ignore_index=True)
    df_subset1 = df_all[df_all['Name'] == s] #Chọn cầu thủ trực tiếp
    position = df_subset1['Position'].iloc[0]
    
    if position == 'Defender':
        df_subset2 = df_subset1[dselected_columns]
    elif position == 'Forward':
        df_subset2 = df_subset1[fselected_columns]
    elif position == 'Goalkeeper':
        df_subset2 = df_subset1[gselected_columns]
    elif position == 'Midfielder':
        df_subset2 = df_subset1[mselected_columns]

    r_values = df_subset2.iloc[0]

    df_radar = pd.DataFrame(dict(
        r=r_values.values,        # Use the aggregated or raw values
        theta=r_values.index      # Use the column names as theta
    ))

    fig = px.line_polar(df_radar, r='r', theta='theta', line_close=True)
    fig.update_traces(fill='toself')
    fig.show()

radar('Oscar Bobb')
  

Players over 25 and under  25 years old by club

In [146]:
df1 = df_defender[df_defender['Age'] > 25] # dataset for players over 25
df2 = df_forward[df_forward['Age'] > 25] # dataset for players over 25
df3 = df_midfielder[df_midfielder['Age'] > 25] # dataset for players over 25
df4 = df_defender[df_defender['Age'] <= 25] # dataset for players under 25
df5 = df_forward[df_forward['Age'] <= 25] # dataset for players under 25
df6 = df_midfielder[df_midfielder['Age'] <= 25] # dataset for players under 25

df_all_players = pd.concat([df1,df2,df3,df4,df5,df6], ignore_index=True)
df_all_over_25 = pd.concat([df1,df2,df3] ,ignore_index= True)
df_all_under_25 = pd.concat([df4, df5, df6], ignore_index=True)

total_playes = df_all_players.groupby('Club').size()

x_under_25 = df_all_under_25.groupby('Club').size()
x_under_25_pro = x_under_25 / total_playes

x_over_25 = df_all_over_25.groupby('Club').size()
x_over_25_pro = x_over_25 / total_playes

y_club_names = np.unique(np.concatenate([df_defender['Club'].unique(),df_forward['Club'].unique(),df_midfielder['Club'].unique()]))

fig = go.Figure()

# Add bars for 'Under 25'
fig.add_trace(go.Bar(
    x=x_under_25_pro,
    y=y_club_names,
    name='Under 25',
    orientation='h',
    marker=dict(color='#59CAB3', line=dict(color='whitesmoke', width=1.1))
))

# Add bars for 'Over 25'
fig.add_trace(go.Bar(
    x=x_over_25_pro,
    y=y_club_names,
    name='Over 25',
    orientation='h',
    marker=dict(color='#4D5FB8', line=dict(color='whitesmoke', width=1.1))
))

# Update layout
fig.update_layout(
    barmode='stack',
    title=dict(
        text="Players over 25 and under 25 years old by club",
        font=dict(size=14)
    ),
    xaxis=dict(title="Proportion"),
    yaxis=dict(title="Club Name", tickfont=dict(size=10)),
    legend=dict(
        title=dict(text="Age of players"),
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="right",
        x=0.95
    ),
    margin=dict(r=30, t=100, b=70),
    height=800,
    width=1000
)

fig.show()

Goalkeepers 

In [147]:
available_features = ['Saves', 'Clean sheets', 'Goals Conceded', 'Penalties Saved',
                      'High Claims', 'Sweeper clearances', 'Errors leading to goal', 'Accurate long balls']
def top10goalkeepers(s):
    df_goalkeeper_saves = df_goalkeeper.sort_values(by=s, ascending=False).head(10)
    fig_saves = px.bar(data_frame=df_goalkeeper_saves, x='Name', y=s,color=s)
    fig_saves.update_traces(marker=dict(line=dict(color='#000000', width=1)))
    fig_saves.update_layout(title = f'Top 10 {s} Premier League')
    fig_saves.show()

for feature in available_features:
    top10goalkeepers(feature)